# Financial Econometrics I: Homework 2

Team Member:

Lin Zhang : 15845542@fsv.cuni.cz

Weiwei Qu : 51014941@fsv.cuni.cz

# Problem 1

##### From the the symbols.csv choose 1 of the 10 Sectors (Industrials, Financials, Health Care, etc.). Download the prices for all the stocks belonging to the corresponding Sector for the period 01/2015 - 12/2021. Exclude the stocks that are not available in the quantmod package. Check that your data contains all the desired symbols (include this check in your output).

##### 

In [ ]:
# Setup environment
Sys.setenv(LANG = "en")
#options(warn = -1)  # suppressing warnings

if (!require(quantmod)) install.packages('quantmod')

library(quantmod)

In [ ]:
# read 377 stock symbols from file
smbs<- as.character(read.csv("symbols2.csv")[,2])
head(smbs)
tail(smbs)


In [ ]:
set.seed(15845542) # seed for random number generation
symbols<-sample(smbs,100) # generate 100 random firms
length(symbols)
symbols

In [ ]:
#download data
data <- lapply(symbols, function(y)
{
    try(getSymbols(y, auto.assign = FALSE,from = as.Date('2019-01-01'), to = '2021-12-31'),silent=TRUE)
})

In [ ]:
names(data) <- symbols
# check all symbols used for downloading process are used, no data is missing

length(data)
# check the date range
head(data[[1]])
tail(data[[1]])

In [ ]:
# filter for Adjusted price
data <- lapply(names(data), function(y){
    data[[y]] <- data[[y]][, paste0(y, '.Adjusted')]
})

# Check data output
lapply(data, head)

##### 1. Compute the logarithmic returns for the time-series of each stock in your dataset.

In [ ]:
# compute log-returns for returns
lrets <- lapply(data, function(y){
  y <- na.omit(diff(log(y)))  
})
head(lrets[[1]])
tail(lrets[[1]])

# Problem 2